In [1]:
%load_ext autoreload
%autoreload 2
import syft as sy
from pymongo import MongoClient
from nacl.signing import SigningKey
from nacl.encoding import HexEncoder
from syft.core.node.common.node_table.user import NoSQLSyftUser

/Users/rasswanth/PySyft/.tox/syft.jupyter/lib/python3.9/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [2]:
key = SigningKey.generate()

In [3]:
user = NoSQLSyftUser(name="Natsu", email="natsu@fairy.tail",budget=5.5,hashed_password="go",salt="lkj",private_key=key,role={"admin":["can_write"]})

In [4]:
user

NoSQLSyftUser(id=<UID: d8157a3bfd894a77b17307197481fd05>, email='natsu@fairy.tail', name='Natsu', budget=5.5, hashed_password='go', salt='lkj', private_key=<nacl.signing.SigningKey object at 0x10793bfa0>, verify_key=None, role={'admin': ['can_write']}, added_by=None, website=None, institution=None, daa_pdf=None, created_at=None)

In [5]:

client = MongoClient(host='localhost', port=49703, username="root", password="example", uuidRepresentation="standard")

In [6]:
db = client["app"]

In [7]:
db.list_collection_names()

['users', 'syft_users']

In [8]:
collection = db["syft_users"]


In [15]:
collection.count_documents(filter={})

6

In [10]:

from syft.core.node.common.node_manager.user_manager import NoSQLUserManager

In [11]:
user_manager = NoSQLUserManager(db)

In [12]:
user.to_mongo()

{'email': 'natsu@fairy.tail',
 '_id': UUID('d8157a3b-fd89-4a77-b173-07197481fd05'),
 '__canonical_name__': 'SyftUser',
 '__version__': 1,
 '__blob__': b'\n\tprotobuf:\x12\x14syft.lib.python.Dict\x1a\xc7\x10\n=\n\tprotobuf:\x12\x16syft.lib.python.String\x1a\x18\n\x02id\x12\x12\n\x10\x9f\xc2WTW\xd1H0\x9b\xd3\x94,\xaa1w\xa1\n@\n\tprotobuf:\x12\x16syft.lib.python.String\x1a\x1b\n\x05email\x12\x12\n\x10$\xbc\x1bW}\xbbA\xe6\xa7\x94?!N\xf3\xf3\x01\n?\n\tprotobuf:\x12\x16syft.lib.python.String\x1a\x1a\n\x04name\x12\x12\n\x10n\xc5\x86\x06\x85\xe9A\xef\xae\x9b\x13!\xc5\xa0@_\nA\n\tprotobuf:\x12\x16syft.lib.python.String\x1a\x1c\n\x06budget\x12\x12\n\x10\xe2kN\x8aV\xe7J\xb5\xa9\xeaA\xf2wQ\x1b\xa7\nJ\n\tprotobuf:\x12\x16syft.lib.python.String\x1a%\n\x0fhashed_password\x12\x12\n\x10\x83\x11\xc6\x95^\xd2A\xf3\xa2\x16\xab\x0c\xees\xf6\xf4\n?\n\tprotobuf:\x12\x16syft.lib.python.String\x1a\x1a\n\x04salt\x12\x12\n\x10\xd2S\xe6\xcc\xcayI\x02\xb9\xd1\xa8\xf0NM\xfa\xcc\nF\n\tprotobuf:\x12\x16syft.lib.pytho

In [13]:
user_manager.add(user)

In [16]:
len(user_manager)

6

In [ ]:
natsu = user_manager.find_one({"email": "natsu@fairy.tail"})

In [ ]:
natsu

In [ ]:
user_manager.drop()

In [ ]:
import syft as sy